# 🤖 RSI Trading Bot - Versión B (Mejorada)

## 📋 Características de Seguridad

- ✅ **Validación de saldo** con margen de seguridad para comisiones
- ✅ **Porcentaje configurable** del capital (default: 10%)
- ✅ **Cooldown entre operaciones** para evitar sobre-trading
- ✅ **Manejo de errores robusto** con logging estructurado
- ✅ **Validación de condiciones** de mercado
- ✅ **Modo testnet** para paper trading seguro

## ⚙️ Configuración

- **Exchange**: Binance Testnet
- **Par**: BTC/USDT
- **Timeframe**: 15m
- **RSI Threshold**: < 30 (sobreventa)
- **Capital por operación**: 10% del saldo disponible

## 📦 1. Instalación de Dependencias

In [ ]:
# Instalación de dependencias necesarias
!pip install ccxt pandas ta python-dotenv -q

## 🔧 2. Imports y Configuración Inicial

In [ ]:
import ccxt
import pandas as pd
import ta
import time
import logging
from datetime import datetime, timedelta
from typing import Optional, Dict, Any
from dataclasses import dataclass
from enum import Enum
import json
from pathlib import Path

# Configuración de logging estructurado
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger('RSI_TradingBot')

## 🏗️ 3. Arquitectura: Clases de Dominio

In [ ]:
class TradingError(Exception):
    """Excepción base para errores de trading"""
    pass

class InsufficientBalanceError(TradingError):
    """Error cuando el saldo es insuficiente"""
    pass

class CooldownActiveError(TradingError):
    """Error cuando el cooldown está activo"""
    pass

class MarketDataError(TradingError):
    """Error al obtener datos de mercado"""
    pass

class OrderExecutionError(TradingError):
    """Error al ejecutar una orden"""
    pass


class OrderStatus(Enum):
    """Estados posibles de una orden"""
    PENDING = "pending"
    EXECUTED = "executed"
    FAILED = "failed"
    COOLDOWN = "cooldown"


@dataclass
class TradingConfig:
    """Configuración del bot de trading"""
    symbol: str = "BTC/USDT"
    timeframe: str = "15m"
    rsi_period: int = 14
    rsi_oversold: float = 30.0
    capital_percentage: float = 0.10  # 10% del saldo
    cooldown_minutes: int = 30
    min_balance_usdt: float = 11.0  # Mínimo para operar (incluye margen para fees)
    fee_percentage: float = 0.001  # 0.1% fee estimado
    testnet: bool = True
    
    def validate(self) -> None:
        """Valida la configuración"""
        if not 0 < self.capital_percentage <= 1:
            raise ValueError("capital_percentage debe estar entre 0 y 1")
        if self.rsi_period < 2:
            raise ValueError("rsi_period debe ser >= 2")
        if self.min_balance_usdt < 10:
            raise ValueError("min_balance_usdt debe ser >= 10 USDT")


@dataclass
class TradeSignal:
    """Señal de trading con contexto completo"""
    timestamp: datetime
    rsi_value: float
    current_price: float
    should_buy: bool
    reason: str
    
    def to_dict(self) -> Dict[str, Any]:
        return {
            'timestamp': self.timestamp.isoformat(),
            'rsi': self.rsi_value,
            'price': self.current_price,
            'signal': 'BUY' if self.should_buy else 'HOLD',
            'reason': self.reason
        }

## 🛡️ 4. Sistema de Validación y Protecciones

In [ ]:
class TradingValidator:
    """Validador de condiciones de trading"""
    
    def __init__(self, config: TradingConfig):
        self.config = config
        self.last_trade_time: Optional[datetime] = None
    
    def validate_balance(self, balance: float, required_amount: float) -> None:
        """
        Valida que el saldo sea suficiente incluyendo margen para fees.
        
        Raises:
            InsufficientBalanceError: Si el saldo es insuficiente
        """
        # Calcular monto total necesario (capital + fees estimados)
        total_needed = required_amount * (1 + self.config.fee_percentage)
        
        if balance < self.config.min_balance_usdt:
            raise InsufficientBalanceError(
                f"Saldo {balance:.2f} USDT por debajo del mínimo "
                f"requerido {self.config.min_balance_usdt:.2f} USDT"
            )
        
        if balance < total_needed:
            raise InsufficientBalanceError(
                f"Saldo insuficiente: {balance:.2f} USDT disponible, "
                f"{total_needed:.2f} USDT necesario (incluye fees)"
            )
        
        logger.info(
            f"✅ Validación de saldo exitosa: {balance:.2f} USDT disponible, "
            f"{total_needed:.2f} USDT requerido"
        )
    
    def validate_cooldown(self) -> None:
        """
        Valida que haya pasado el tiempo de cooldown desde la última operación.
        
        Raises:
            CooldownActiveError: Si el cooldown está activo
        """
        if self.last_trade_time is None:
            return
        
        cooldown_delta = timedelta(minutes=self.config.cooldown_minutes)
        time_since_last_trade = datetime.now() - self.last_trade_time
        
        if time_since_last_trade < cooldown_delta:
            remaining = cooldown_delta - time_since_last_trade
            raise CooldownActiveError(
                f"Cooldown activo. Tiempo restante: {remaining.seconds // 60} minutos"
            )
        
        logger.info("✅ Cooldown completado, listo para operar")
    
    def mark_trade_executed(self) -> None:
        """Marca el timestamp de la última operación"""
        self.last_trade_time = datetime.now()
        logger.info(f"⏰ Cooldown iniciado hasta {self.last_trade_time + timedelta(minutes=self.config.cooldown_minutes)}")
    
    def validate_market_data(self, df: pd.DataFrame) -> None:
        """
        Valida que los datos de mercado sean suficientes y válidos.
        
        Raises:
            MarketDataError: Si los datos son insuficientes o inválidos
        """
        if df is None or df.empty:
            raise MarketDataError("DataFrame de mercado vacío")
        
        if len(df) < self.config.rsi_period + 1:
            raise MarketDataError(
                f"Datos insuficientes: {len(df)} velas, "
                f"se requieren al menos {self.config.rsi_period + 1}"
            )
        
        required_columns = ['open', 'high', 'low', 'close', 'volume']
        missing = [col for col in required_columns if col not in df.columns]
        if missing:
            raise MarketDataError(f"Columnas faltantes: {missing}")
        
        if df['close'].isna().any():
            raise MarketDataError("Datos de precio contienen valores NaN")
        
        logger.info(f"✅ Datos de mercado válidos: {len(df)} velas disponibles")

## 🤖 5. Bot de Trading Principal

In [ ]:
class RSITradingBot:
    """Bot de trading basado en RSI con protecciones de seguridad"""
    
    def __init__(self, config: TradingConfig, api_key: str = None, api_secret: str = None):
        config.validate()
        self.config = config
        self.validator = TradingValidator(config)
        self.exchange = self._initialize_exchange(api_key, api_secret)
        self.trade_history = []
        
        logger.info("="*80)
        logger.info("🚀 RSI Trading Bot Inicializado")
        logger.info(f"📊 Símbolo: {config.symbol}")
        logger.info(f"⏱️  Timeframe: {config.timeframe}")
        logger.info(f"📉 RSI Oversold: < {config.rsi_oversold}")
        logger.info(f"💰 Capital por operación: {config.capital_percentage * 100}%")
        logger.info(f"⏳ Cooldown: {config.cooldown_minutes} minutos")
        logger.info(f"🧪 Modo: {'TESTNET' if config.testnet else 'PRODUCCIÓN'}")
        logger.info("="*80)
    
    def _initialize_exchange(self, api_key: Optional[str], api_secret: Optional[str]) -> ccxt.Exchange:
        """Inicializa la conexión con Binance"""
        try:
            exchange = ccxt.binance({
                'apiKey': api_key,
                'secret': api_secret,
                'enableRateLimit': True,
                'options': {
                    'defaultType': 'spot',
                }
            })
            
            if self.config.testnet:
                exchange.set_sandbox_mode(True)
                logger.info("🧪 Modo TESTNET activado")
            
            # Verificar conectividad
            exchange.load_markets()
            logger.info(f"✅ Conectado a Binance {'Testnet' if self.config.testnet else 'Mainnet'}")
            
            return exchange
            
        except Exception as e:
            logger.error(f"❌ Error al conectar con Binance: {e}")
            raise TradingError(f"Fallo en inicialización del exchange: {e}")
    
    def fetch_market_data(self) -> pd.DataFrame:
        """
        Obtiene datos OHLCV del mercado.
        
        Returns:
            DataFrame con datos de mercado
            
        Raises:
            MarketDataError: Si falla la obtención de datos
        """
        try:
            # Obtener más velas de las necesarias para cálculo preciso de RSI
            limit = self.config.rsi_period * 3
            
            logger.info(f"📥 Obteniendo {limit} velas de {self.config.symbol} ({self.config.timeframe})...")
            
            ohlcv = self.exchange.fetch_ohlcv(
                symbol=self.config.symbol,
                timeframe=self.config.timeframe,
                limit=limit
            )
            
            df = pd.DataFrame(
                ohlcv,
                columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
            )
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            
            self.validator.validate_market_data(df)
            
            logger.info(f"✅ Datos obtenidos: {len(df)} velas desde {df['timestamp'].iloc[0]}")
            return df
            
        except ccxt.NetworkError as e:
            raise MarketDataError(f"Error de red al obtener datos: {e}")
        except ccxt.ExchangeError as e:
            raise MarketDataError(f"Error del exchange: {e}")
        except Exception as e:
            raise MarketDataError(f"Error inesperado al obtener datos: {e}")
    
    def calculate_rsi(self, df: pd.DataFrame) -> float:
        """
        Calcula el RSI usando la librería ta.
        
        Returns:
            Valor actual del RSI
        """
        rsi_indicator = ta.momentum.RSIIndicator(
            close=df['close'],
            window=self.config.rsi_period
        )
        df['rsi'] = rsi_indicator.rsi()
        
        current_rsi = df['rsi'].iloc[-1]
        logger.info(f"📊 RSI actual: {current_rsi:.2f}")
        
        return current_rsi
    
    def generate_signal(self, df: pd.DataFrame) -> TradeSignal:
        """
        Genera señal de trading basada en RSI.
        
        Returns:
            TradeSignal con la decisión y contexto
        """
        rsi = self.calculate_rsi(df)
        current_price = df['close'].iloc[-1]
        
        should_buy = rsi < self.config.rsi_oversold
        
        if should_buy:
            reason = f"RSI en sobreventa ({rsi:.2f} < {self.config.rsi_oversold})"
        else:
            reason = f"RSI no indica sobreventa ({rsi:.2f} >= {self.config.rsi_oversold})"
        
        signal = TradeSignal(
            timestamp=datetime.now(),
            rsi_value=rsi,
            current_price=current_price,
            should_buy=should_buy,
            reason=reason
        )
        
        logger.info(f"🎯 Señal generada: {signal.to_dict()}")
        return signal
    
    def get_available_balance(self) -> float:
        """
        Obtiene el saldo disponible de USDT.
        
        Returns:
            Saldo disponible en USDT
            
        Raises:
            TradingError: Si falla la obtención del saldo
        """
        try:
            balance = self.exchange.fetch_balance()
            usdt_balance = balance['free']['USDT']
            
            logger.info(f"💰 Saldo disponible: {usdt_balance:.2f} USDT")
            return usdt_balance
            
        except Exception as e:
            raise TradingError(f"Error al obtener saldo: {e}")
    
    def calculate_order_amount(self, balance: float, current_price: float) -> Dict[str, float]:
        """
        Calcula el monto de la orden basado en el porcentaje configurado.
        
        Returns:
            Dict con 'usdt_amount' y 'btc_amount'
        """
        usdt_to_use = balance * self.config.capital_percentage
        btc_amount = usdt_to_use / current_price
        
        logger.info(
            f"📊 Cálculo de orden: {usdt_to_use:.2f} USDT "
            f"({self.config.capital_percentage * 100}% de {balance:.2f}) "
            f"= {btc_amount:.6f} BTC @ {current_price:.2f}"
        )
        
        return {
            'usdt_amount': usdt_to_use,
            'btc_amount': btc_amount
        }
    
    def execute_market_buy(self, amount: float, price: float) -> Dict[str, Any]:
        """
        Ejecuta una orden de compra de mercado.
        
        Args:
            amount: Cantidad de BTC a comprar
            price: Precio actual (para logging)
            
        Returns:
            Información de la orden ejecutada
            
        Raises:
            OrderExecutionError: Si falla la ejecución
        """
        try:
            logger.info(f"🔄 Ejecutando orden de compra: {amount:.6f} BTC...")
            
            order = self.exchange.create_market_buy_order(
                symbol=self.config.symbol,
                amount=amount
            )
            
            logger.info(f"✅ Orden ejecutada exitosamente")
            logger.info(f"   Order ID: {order['id']}")
            logger.info(f"   Status: {order['status']}")
            logger.info(f"   Filled: {order.get('filled', 'N/A')} BTC")
            logger.info(f"   Cost: {order.get('cost', 'N/A')} USDT")
            
            return order
            
        except ccxt.InsufficientFunds as e:
            raise OrderExecutionError(f"Fondos insuficientes: {e}")
        except ccxt.InvalidOrder as e:
            raise OrderExecutionError(f"Orden inválida: {e}")
        except ccxt.NetworkError as e:
            raise OrderExecutionError(f"Error de red durante ejecución: {e}")
        except Exception as e:
            raise OrderExecutionError(f"Error inesperado al ejecutar orden: {e}")
    
    def run_trading_cycle(self) -> Dict[str, Any]:
        """
        Ejecuta un ciclo completo de trading con todas las validaciones.
        
        Returns:
            Resultado del ciclo con status y detalles
        """
        result = {
            'timestamp': datetime.now().isoformat(),
            'status': OrderStatus.PENDING.value,
            'signal': None,
            'order': None,
            'error': None
        }
        
        try:
            logger.info("\n" + "="*80)
            logger.info("🔄 INICIANDO CICLO DE TRADING")
            logger.info("="*80)
            
            # 1. Validar cooldown
            logger.info("\n[1/6] Validando cooldown...")
            self.validator.validate_cooldown()
            
            # 2. Obtener datos de mercado
            logger.info("\n[2/6] Obteniendo datos de mercado...")
            df = self.fetch_market_data()
            
            # 3. Generar señal
            logger.info("\n[3/6] Analizando señal de trading...")
            signal = self.generate_signal(df)
            result['signal'] = signal.to_dict()
            
            if not signal.should_buy:
                logger.info(f"⏸️  No hay señal de compra: {signal.reason}")
                result['status'] = OrderStatus.PENDING.value
                return result
            
            logger.info(f"✅ Señal de COMPRA detectada: {signal.reason}")
            
            # 4. Obtener y validar saldo
            logger.info("\n[4/6] Validando saldo...")
            balance = self.get_available_balance()
            order_amounts = self.calculate_order_amount(balance, signal.current_price)
            
            self.validator.validate_balance(balance, order_amounts['usdt_amount'])
            
            # 5. Ejecutar orden
            logger.info("\n[5/6] Ejecutando orden de compra...")
            order = self.execute_market_buy(
                amount=order_amounts['btc_amount'],
                price=signal.current_price
            )
            
            result['order'] = {
                'id': order['id'],
                'status': order['status'],
                'filled': order.get('filled'),
                'cost': order.get('cost')
            }
            
            # 6. Marcar cooldown
            logger.info("\n[6/6] Activando cooldown...")
            self.validator.mark_trade_executed()
            
            result['status'] = OrderStatus.EXECUTED.value
            self.trade_history.append(result)
            
            logger.info("\n" + "="*80)
            logger.info("✅ CICLO COMPLETADO EXITOSAMENTE")
            logger.info("="*80 + "\n")
            
        except CooldownActiveError as e:
            logger.warning(f"⏳ {e}")
            result['status'] = OrderStatus.COOLDOWN.value
            result['error'] = str(e)
            
        except InsufficientBalanceError as e:
            logger.error(f"💸 {e}")
            result['status'] = OrderStatus.FAILED.value
            result['error'] = str(e)
            
        except (MarketDataError, OrderExecutionError) as e:
            logger.error(f"❌ {e}")
            result['status'] = OrderStatus.FAILED.value
            result['error'] = str(e)
            
        except Exception as e:
            logger.exception(f"💥 Error inesperado: {e}")
            result['status'] = OrderStatus.FAILED.value
            result['error'] = f"Error inesperado: {e}"
        
        return result
    
    def get_trade_history(self) -> list:
        """Retorna el historial de operaciones"""
        return self.trade_history
    
    def save_trade_history(self, filepath: str = "trade_history.json") -> None:
        """Guarda el historial de operaciones en un archivo JSON"""
        with open(filepath, 'w') as f:
            json.dump(self.trade_history, f, indent=2)
        logger.info(f"💾 Historial guardado en {filepath}")

## 🔐 6. Configuración de Credenciales

⚠️ **IMPORTANTE**: Nunca hardcodees tus API keys. Usa variables de entorno o archivos `.env`.

In [ ]:
# Opción 1: Cargar desde archivo .env (recomendado)
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv('BINANCE_TESTNET_API_KEY', '')
API_SECRET = os.getenv('BINANCE_TESTNET_API_SECRET', '')

# Opción 2: Input manual (solo para testing)
if not API_KEY or not API_SECRET:
    print("⚠️  Credenciales no encontradas en .env")
    print("\nPuedes obtener credenciales de testnet en:")
    print("https://testnet.binance.vision/")
    
    # Descomentar para input manual (NO RECOMENDADO EN PRODUCCIÓN)
    # API_KEY = input("API Key: ")
    # API_SECRET = input("API Secret: ")

## 🚀 7. Inicialización y Ejecución del Bot

In [ ]:
# Configuración del bot
config = TradingConfig(
    symbol="BTC/USDT",
    timeframe="15m",
    rsi_period=14,
    rsi_oversold=30.0,
    capital_percentage=0.10,  # 10% del saldo
    cooldown_minutes=30,
    min_balance_usdt=11.0,
    testnet=True
)

# Inicializar bot
bot = RSITradingBot(config, api_key=API_KEY, api_secret=API_SECRET)

## 🔄 8. Ejecutar Ciclo Único de Trading

In [ ]:
# Ejecutar un ciclo de trading
result = bot.run_trading_cycle()

# Mostrar resultado
print("\n" + "="*80)
print("📊 RESULTADO DEL CICLO")
print("="*80)
print(json.dumps(result, indent=2))
print("="*80)

## 🔁 9. Loop Continuo (Opcional)

⚠️ **Advertencia**: Esto ejecutará el bot indefinidamente. Usa con precaución.

In [ ]:
# DESCOMENTAR PARA EJECUTAR EN LOOP CONTINUO

# import time

# CHECK_INTERVAL_SECONDS = 60  # Revisar cada 60 segundos
# MAX_ITERATIONS = 100  # Límite de seguridad

# logger.info(f"🔁 Iniciando loop continuo (intervalo: {CHECK_INTERVAL_SECONDS}s)")
# logger.info(f"⏹️  Presiona Ctrl+C para detener\n")

# try:
#     for i in range(MAX_ITERATIONS):
#         logger.info(f"\n{'='*80}")
#         logger.info(f"Iteración {i+1}/{MAX_ITERATIONS}")
#         logger.info(f"{'='*80}")
#         
#         result = bot.run_trading_cycle()
#         
#         logger.info(f"\n⏳ Esperando {CHECK_INTERVAL_SECONDS} segundos...\n")
#         time.sleep(CHECK_INTERVAL_SECONDS)
#         
# except KeyboardInterrupt:
#     logger.info("\n⏹️  Bot detenido por el usuario")
# finally:
#     bot.save_trade_history()
#     logger.info("✅ Historial guardado")

## 📊 10. Análisis de Resultados

In [ ]:
# Ver historial de operaciones
history = bot.get_trade_history()

if history:
    print(f"\n📈 Total de operaciones ejecutadas: {len(history)}\n")
    
    for i, trade in enumerate(history, 1):
        print(f"\n{'='*60}")
        print(f"Operación #{i}")
        print(f"{'='*60}")
        print(json.dumps(trade, indent=2))
else:
    print("\n📭 No hay operaciones en el historial")

## 🧪 11. Testing de Componentes Individuales

In [ ]:
# Test 1: Obtener datos de mercado
print("\n🧪 Test 1: Obtención de datos de mercado")
print("="*60)
try:
    df = bot.fetch_market_data()
    print(f"✅ Datos obtenidos: {len(df)} velas")
    print(f"\nÚltimas 5 velas:")
    print(df[['timestamp', 'close', 'volume']].tail())
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test 2: Cálculo de RSI
print("\n🧪 Test 2: Cálculo de RSI")
print("="*60)
try:
    df = bot.fetch_market_data()
    rsi = bot.calculate_rsi(df)
    print(f"✅ RSI actual: {rsi:.2f}")
    print(f"   Umbral de sobreventa: {config.rsi_oversold}")
    print(f"   Estado: {'🔴 SOBREVENTA' if rsi < config.rsi_oversold else '🟢 NORMAL'}")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test 3: Verificar saldo
print("\n🧪 Test 3: Verificación de saldo")
print("="*60)
try:
    balance = bot.get_available_balance()
    print(f"✅ Saldo disponible: {balance:.2f} USDT")
    print(f"   Mínimo requerido: {config.min_balance_usdt:.2f} USDT")
    print(f"   Capital por operación ({config.capital_percentage*100}%): {balance * config.capital_percentage:.2f} USDT")
    print(f"   Estado: {'✅ SUFICIENTE' if balance >= config.min_balance_usdt else '❌ INSUFICIENTE'}")
except Exception as e:
    print(f"❌ Error: {e}")

In [ ]:
# Test 4: Generar señal completa
print("\n🧪 Test 4: Generación de señal")
print("="*60)
try:
    df = bot.fetch_market_data()
    signal = bot.generate_signal(df)
    print(f"✅ Señal generada:")
    print(json.dumps(signal.to_dict(), indent=2))
except Exception as e:
    print(f"❌ Error: {e}")

## 📝 12. Notas y Mejoras Futuras

### ✅ Implementado
- Validación de saldo con margen para fees
- Sistema de cooldown entre operaciones
- Logging estructurado y detallado
- Manejo de errores tipado por capas
- Configuración centralizada y validada
- Protección contra sobre-trading

### 🚀 Mejoras Sugeridas
1. **Stop Loss Automático**: Implementar órdenes de stop loss después de cada compra
2. **Take Profit**: Definir niveles de salida basados en % de ganancia
3. **Confirmación Multi-Indicador**: Agregar MACD, Bollinger Bands para confirmar señales
4. **Backtesting**: Sistema para probar la estrategia con datos históricos
5. **Notificaciones**: Integrar Telegram/Discord para alertas en tiempo real
6. **Base de Datos**: Persistir historial en PostgreSQL/MongoDB
7. **Dashboard**: Visualización en tiempo real con Plotly/Streamlit
8. **Position Sizing Dinámico**: Ajustar % según volatilidad (ATR)
9. **Trailing Stop**: Stop loss que sigue el precio al alza
10. **Circuit Breaker**: Detener bot después de N pérdidas consecutivas

### ⚠️ Recordatorios de Seguridad
- **NUNCA** uses 100% del capital en producción
- **SIEMPRE** prueba en testnet primero
- **REVISA** los logs antes de cada sesión
- **MONITOREA** las operaciones activamente
- **ACTUALIZA** las API keys regularmente